# Assignment 4 - Riconoscimento tramite Deep Learning

Requisiti:
- Rifacimento assignment sul riconoscimento, questa volta utilizzando però una rete neurale profonda.

Per questa esercitazioni, le scelte tecniche sono state:
- Yolov8 per la face detection nelle immagini e video
- Classificazione tramite CNN profonda in Tensorflow

Bonus:
- Utilizzo di una label extra per i volti sconosciuti

In [1]:
import os
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2 as cv
import pickle
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Rescaling, Resizing, RandomFlip, RandomRotation, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.losses import SparseCategoricalCrossentropy 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import accuracy_score

Il primo step consiste nel far scorrere il nostro face detector lungo tutte le immagini già acquisite, farne il rescale per renderle quanto più simili possibili alle immagini senza una identità precisa (che da ora in avanti definiremo Unknown) e fare data augmentation per aumentare i nostri dati a disposizione.

Per la data augmentation, in particolare, sono state applicate trasformazioni randomizzate:
- flip orizzontale dell'immagine
- leggera rotazione (fattore 0.2)

In [2]:
# Inizializziamo Yolov8 e le immagini per fare training, test e validazione

train = None
test = None
val = None
yolo = YOLO('yolov8n-face.pt')

Imgs_train = Imgs_test = Imgs_val = None
Labels_train = Labels_test = Labels_val = None

if os.path.exists('train.pkl'):
    trainfile = open('train.pkl', 'rb')
    testfile = open('test.pkl', 'rb')
    valfile = open('val.pkl', 'rb')

    train = pickle.load(trainfile) 
    test = pickle.load(testfile)
    val = pickle.load(valfile)

    Imgs_train = train['imgs']
    Imgs_test = test['imgs']
    Imgs_val = val['imgs']

    Labels_train = train['labels']
    Labels_test = test['labels']
    Labels_val = val['labels']

Per le dimensioni di train-test-validation, abbiamo suddiviso le immagini etichettate precedentemente ottenute con le seguenti percentuali, rispettivamente 60-20-20.

In [3]:
# Processo di face detection dal nostro dataset di circa 30k foto (compresi volti etichettati come Unknown, non facenti parte del nostro pool di 4 identità note)

resize_and_rescale = Sequential([
        Resizing(64, 64),
        Rescaling(1./255)
    ])

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
])

if not os.path.exists('train.pkl'):
    root = 'train'
    paths = []
    labels = []

    for dir in os.listdir(root):
        tmp = glob.glob(f'{root}/{dir}/*.jpg')
        labels.extend(len(tmp) * [dir])
        paths.extend(tmp)

    imgs = []
    img_labels = []

    # Sfruttiamo la funzionalità di yolo che ci permette di andare a fare 
    # face detection considerando solo le bbox con label 0, cioè con classe 'person'.
    # Andiamo inoltre a prendere soltanto la bbox la cui label ha la confidenza più alta;
    # Il motivo di ciò è che durante i test è emerso che anche i volti riflessi in eventuali
    # finestre venivano rilevati, dando vita a una sorta di falso positivo (ma con comunque)
    # una confidenza minore rispetto al volto "reale". Facciamo infine il crop dell'immagine
    # per andare a estrarre solo la porzione desiderata, usando due piccoli modelli per 
    # fare resize e rescale delle immagini per normalizzarle e per fare data augmentation,
    # applicando una piccola rotazione casuale e un flip orizzontale randomico

    for i in range(len(paths)):
        path = paths[i]
        img = cv.imread(path)
        result = yolo(img, classes=0, verbose=False)[0].numpy()
        if len(result) != 0:
            boxes = result.boxes
            conf = boxes.conf
            argmax = np.argmax(conf)
            box = boxes[argmax]
            xyxy = box.xyxy[0]
            xyxy = [round(xy) for xy in xyxy]
            crop = img[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]

            crop = resize_and_rescale(crop).numpy()
            crop_aug = data_augmentation(crop).numpy()

            imgs.append(crop)
            imgs.append(crop_aug)
            img_labels.append(labels[i])
            img_labels.append(labels[i])

    # Procediamo a dividere le nostre immagini etichettate in 3 insiemi per fare poi
    # train, test e validation. Le dimensioni di questi saranno rispettivamente il
    # 60%, 20% e 20% del totale delle immagini a disposizione.
    # Salviamo infine tramite pickle il lavoro svolto per evitare sprechi di tempo futuri

    Imgs_train, Imgs_test, Labels_train, Labels_test = train_test_split(imgs, img_labels, test_size=0.4)
    Imgs_val, Imgs_test, Labels_val, Labels_test = train_test_split(Imgs_test, Labels_test, test_size=0.5)
    
    Imgs_train = np.asarray(Imgs_train)
    Imgs_test = np.asarray(Imgs_test)
    Imgs_val = np.asarray(Imgs_val)
    
    Labels_train = LabelEncoder().fit_transform(Labels_train)
    Labels_test = LabelEncoder().fit_transform(Labels_test)
    Labels_val = LabelEncoder().fit_transform(Labels_val)

    trainfile = open('train.pkl', 'wb')
    testfile = open('test.pkl', 'wb')
    valfile = open('val.pkl', 'wb')

    train = {
        "imgs" : Imgs_train,
        "labels" : Labels_train
    }

    test = {
        "imgs" : Imgs_test,
        "labels" : Labels_test
    }

    val = {
        "imgs" : Imgs_val,
        "labels" : Labels_val
    }

    pickle.dump(train, trainfile)
    pickle.dump(test, testfile)
    pickle.dump(val, valfile)

    trainfile.close()
    testfile.close()
    valfile.close()

In [4]:
imgShape = (64, 64, 3)
num_classes = 5

Sono stati effettuati diversi tentativi al fine di scegliere il modello migliore per il nostro compito, variando sia iperparametri che struttura della rete, avvalendosi anche in alcuni tentativi di regolarizzazione l2 dei pesi in quanto, durante le prime prove, il problema più evidente era un overfitting in fase di test, che si traduceva poi in accuracy estremamente basse in fase di test.

Per evitare sprechi di tempo e risorse, è stato utile aggiungere alla rete un meccanismo di early stopping mediamente sensibile sull'accuracy, nel caso in cui si arrivasse ad una situazione piatta e non riuscisse più ad aumentare dopo alcune epoche. 

In [5]:
model_name = 'muzio_2.keras'

if os.path.exists(model_name):
  model = load_model(model_name)
else:

  model = Sequential([
    Conv2D(64, 3, activation='relu', padding='same', input_shape=imgShape),
    MaxPooling2D((2, 2)),
    Conv2D(128, 3, activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, 3, activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(num_classes, activation='softmax')
  ])

  model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

  early_stop = EarlyStopping(monitor='accuracy', patience = 3, restore_best_weights = True)

  hist = model.fit(Imgs_train, 
                  Labels_train, 
                  epochs=100,
                  batch_size=64,
                  validation_data=(Imgs_val, Labels_val),
                  callbacks=[early_stop]
                  )
  
  model.save('muzio.keras')

Estraiamo 20 frame e annotiamoli manualmente per fare model selection: questi 20 frame non andranno a coincidere ovviamente con la porzione del video usata per i test.

In [6]:
classes_ = ['Davide', 'Francesco', 'Gabriele', 'Stefano', 'Unknown']

annotation = None
font = cv.FONT_HERSHEY_SIMPLEX

if os.path.exists('annotateset.pkl'):
    annotationfile = open('annotateset.pkl', 'rb')
    annotation = pickle.load(annotationfile)
    annotationfile.close()
else:
    annotation = []
    cap = cv.VideoCapture('Video finale senza riconoscimento.mp4')
    ANNOTATION_FRAMES = 20
    output_annote = None
    f = 0
    while cap.isOpened() and f < ANNOTATION_FRAMES:
        f += 1
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        if output_annote is None:
            height, width, channels = frame.shape
            size = (width,height)
            output_annote = cv.VideoWriter('manual.mp4', -1, 2, size)

        faces = yolo(frame, verbose=False)[0]

        faces = faces.numpy()

        frame_faces = []

        intact = frame.copy()

        for box in faces.boxes:
            tmp = intact.copy()
            
            xyxy = box.xyxy[0]
            xyxy = [round(xy) for xy in xyxy]

            tmp = cv.rectangle(tmp, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)

            cv.imshow('', tmp)
            cv.waitKey(0)
            cv.destroyAllWindows()

            # Annoto manualmente ogni bounding box nell'immagine con la vera identità (Davide,Stefano,Gabriele,Francesco o Unknown)

            frame_face = input()
            frame_faces.append(frame_face)

            frame = cv.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)
            frame = cv.putText(frame, frame_face, (xyxy[0], xyxy[1]), font, 1, (0, 0, 255), 1, cv.LINE_AA, False)
        annotation.append(frame_faces)
        output_annote.write(frame)
    cap.release()
    output_annote.release()

    annotatefile = open('annotateset.pkl','wb')
    pickle.dump(annotation, annotatefile)
    annotatefile.close()

In [7]:
cap = cv.VideoCapture('Video finale senza riconoscimento.mp4')
MAX_FRAME = 20
i = 0
output = None

test_data = []

while cap.isOpened() and i < MAX_FRAME:
    i += 1
    if True:
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        if output is None:
            height, width, channels = frame.shape
            size = (width,height)
            output = cv.VideoWriter('deep_auto_test.mp4', -1, 1, size)

        faces = yolo(frame, verbose=False)[0].numpy()

        frame_data = []

        if len(faces) != 0:
            boxes = faces.boxes

            for box in boxes:
                xyxy = box.xyxy[0]
                xyxy = [round(xy) for xy in xyxy]
                crop = np.array([frame[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]])
                crop = resize_and_rescale(crop)

                pred = model.predict(crop, verbose=False)
                conf = np.max(pred)
                class_ = classes_[np.argmax(pred)]

                frame_data.append(class_)

                frame = cv.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)
                frame = cv.putText(frame, class_, (xyxy[0], xyxy[1]), font, 1, (0, 0, 255), 1, cv.LINE_AA, False)

            test_data.append(frame_data)

            output.write(frame)
                
cap.release()
output.release()

Calcoliamo l'accuracy del nostro modello: dal momento che il face detector è lo stesso, ci basterà usare come metro di misura la somiglianza delle classi assegnate a ogni volto in ogni frame.

In [8]:
annotation_enc= [LabelEncoder().fit_transform(x) for x in annotation]
test_data_enc = [LabelEncoder().fit_transform(x) for x in test_data]

acc = 0

for annotation_item, test_item in zip(annotation_enc, test_data_enc):
    score = accuracy_score(annotation_item, test_item) 
    acc += score

acc /= len(annotation_enc)

In [9]:
print(acc)

0.29166666666666663


Andiamo adesso ad analizzare un video (che non ha overlap con la porzione usata per la validazione) per testare i risultati effettivi del nostro modello.

In [10]:
cap = cv.VideoCapture('Video finale senza riconoscimento.mp4')
MAX_FRAME = 2000
i = 0
output = None

test_data = []

while cap.isOpened() and i < MAX_FRAME:
    i += 1
    if i > 20:
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        if output is None:
            height, width, channels = frame.shape
            size = (width,height)
            output = cv.VideoWriter('deep_auto.mp4', -1, 15, size)

        faces = yolo(frame, verbose=False)[0].numpy()

        frame_data = []

        if len(faces) != 0:
            boxes = faces.boxes

            for box in boxes:
                xyxy = box.xyxy[0]
                xyxy = [round(xy) for xy in xyxy]
                crop = np.array([frame[xyxy[1] : xyxy[3], xyxy[0] : xyxy[2]]])
                crop = resize_and_rescale(crop)

                pred = model.predict(crop, verbose=False)
                conf = np.max(pred)
                class_ = classes_[np.argmax(pred)]

                frame_data.append(class_)

                frame = cv.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 1)
                frame = cv.putText(frame, class_, (xyxy[0], xyxy[1]), font, 1, (0, 0, 255), 1, cv.LINE_AA, False)

            test_data.append(frame_data)

            output.write(frame)
                
cap.release()
output.release()

## Misure di accuracy e considerazioni finali

Sono stati testati 7 diversi modelli, con strutture e iperparametri più o meno diversi. Tuttavia i tentativi non hanno portato a risultati con accuracy soddisfacente. Potrebbe essere quindi ragionevole, a valle di tutto il lavoro svolto, dare molto più peso alla fase di raccolta, pulizia e selezione eventuale dei dati, cercando di catturare i volti scelti in condizioni più varie e tenendo conto di fattori come la presenza o meno degli occhiali, dal momento che il modello sembra comportarsi bene nel momento in cui sono stati fatti test di classificazioni usando le immagini di test del dataset di partenza, ma meno bene nel momento in cui le condizioni sono cambiate ed è stato usato un video con un setup leggermente diverso.

Dal momento che ha dimostrato un'accuracy maggiore, andremo ad utilizzare il modello muzio_3.keras

A seguire, la precisione ottenuta e la summary di ogni modello

|   Model   |   Accuracy   |
|-----------|--------------|
|  Muzio 0  |      1%      |
|  Muzio 1  |      23%     |
|  Muzio 2  |      29%     |
|  Muzio 3  |      15%     |
|  Muzio 4  |      7%      |
|  Muzio 5  |      5%      |
|  Muzio 6  |      14%     |

In [11]:
model_ = load_model('muzio_0.keras')

model_.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                                 
 flatten (Flatten)           (None, 16384)             0

In [12]:
model_ = load_model('muzio_1.keras')

model_.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 8, 8, 64)         

In [13]:
model_ = load_model('muzio_2.keras')

model_.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 6, 6, 128)        

In [14]:
model_ = load_model('muzio_3.keras')

model_.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 6, 6, 128)       

In [15]:
model_ = load_model('muzio_4.keras')

model_.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 60, 60, 64)        4864      
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 6, 6, 128)       

In [16]:
model_ = load_model('muzio_5.keras')

model_.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       204928    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 5, 5, 128)         0

In [17]:
model_ = load_model('muzio_6.keras')

model_.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 256)       295168    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 8, 8, 256)        